# Infrastructure

In [1]:
# Import packages
import json
import pandas as pd
import numpy as np
from pyproj import Transformer, Geod
from tqdm import tqdm
# Ignore copy warning
pd.options.mode.chained_assignment = None 

# Infrastructure point data  

In [2]:
# Load JSON point data files 

with open('london_infrastructure/cycle_parking.json', 'r') as f:
    cycle_parking_json = json.loads(f.read())
cycle_parking = pd.json_normalize(cycle_parking_json, record_path = ['features'])

with open('london_infrastructure/restricted_point.json', 'r') as f:
    restricted_point_json = json.loads(f.read())
restricted_point = pd.json_normalize(restricted_point_json, record_path = ['features'])

with open('london_infrastructure/signage.json', 'r') as f:
    signage_json = json.loads(f.read())
signage = pd.json_normalize(signage_json, record_path = ['features'])

with open('london_infrastructure/signal.json', 'r') as f:
    signal_json = json.loads(f.read())
signal = pd.json_normalize(signal_json, record_path = ['features'])

with open('london_infrastructure/traffic_calming.json', 'r') as f:
    traffic_calming_json = json.loads(f.read())
traffic_calming = pd.json_normalize(traffic_calming_json, record_path = ['features'])

In [3]:
# Save as csv files 
cycle_parking.to_csv('london_infrastructure/cycle_parking.csv', index=False)
restricted_point.to_csv('london_infrastructure/restricted_point.csv', index=False)
signage.to_csv('london_infrastructure/signage.csv', index=False)
signal.to_csv('london_infrastructure/signal.csv', index=False)
traffic_calming.to_csv('london_infrastructure/traffic_calming.csv', index=False)

In [4]:
# Load new csv files
cycle_parking = pd.read_csv('london_infrastructure/cycle_parking.csv')
restricted_point = pd.read_csv('london_infrastructure/restricted_point.csv')
signage = pd.read_csv('london_infrastructure/signage.csv', low_memory=False)
signal = pd.read_csv('london_infrastructure/signal.csv')
traffic_calming = pd.read_csv('london_infrastructure/traffic_calming.csv')

In [5]:
# Function to process dataframes 

def process_file(file):
    
    # Drop unnecessary columns, rename remaining columns
    file.drop(['type', 'geometry.type', 'properties.FEATURE_ID', 'properties.PHOTO1_URL', 'properties.PHOTO2_URL'],
              axis=1, inplace=True)
    file.rename(columns={'geometry.coordinates':'coordinates', 'properties.SVDATE':'date_of_survey', 
                         'properties.BOROUGH':'borough'}, inplace=True)
    
    # Clean coordinate data
    file['coordinates'] = file['coordinates'].str.replace('[\[\]]', '', regex=True)
    file['latitude'] = [file['coordinates'][i].split(', ')[1] for i in range(file.shape[0])]
    file['longitude'] = [file['coordinates'][i].split(', ')[0] for i in range(file.shape[0])]
    
    # Remove duplicate points (keep most recent)
    file.sort_values(by='date_of_survey', ascending=True, inplace=True)
    file.drop_duplicates(subset=['latitude', 'longitude'], inplace=True, keep='last')
    
    # Return clean file  
    file.drop(['coordinates', 'date_of_survey'], axis=1, inplace=True)
    file.reset_index(drop=True, inplace=True)
    return file

In [6]:
# Process dataframes 
restricted_point = process_file(restricted_point)
signal = process_file(signal)
traffic_calming = process_file(traffic_calming)
cycle_parking = process_file(cycle_parking)
signage = process_file(signage)

In [7]:
# Process data further 

# Drop columns with majority null values 
signage.drop(['properties.SS_ROUTEN', 'properties.SS_ACCESS'], axis=1, inplace=True)

# Correct typo in True/False column  
signage['properties.SS_CYCSMB'] = np.where(signage['properties.SS_CYCSMB']=='TRUE', True, False)

# Set unknown or unexpected values to null  
signage['properties.SS_COLOUR'] = signage['properties.SS_COLOUR'].replace('<Null>', np.nan)
signage['properties.SS_NAME'] = signage['properties.SS_NAME'].replace([' ', 'UNKNOWN', 'TRUE'], np.nan)

In [8]:
# Save final versions 
cycle_parking.to_csv('london_infrastructure/cycle_parking.csv', index=False)
restricted_point.to_csv('london_infrastructure/restricted_point.csv', index=False)
signage.to_csv('london_infrastructure/signage.csv', index=False)
signal.to_csv('london_infrastructure/signal.csv', index=False)
traffic_calming.to_csv('london_infrastructure/traffic_calming.csv', index=False)

# Infrastructure line data  

In [9]:
# Load JSON line data files 

with open('london_infrastructure/advanced_stop_line.json', 'r') as f:
    advanced_stop_line_json = json.loads(f.read())
advanced_stop_line = pd.json_normalize(advanced_stop_line_json, record_path = ['features'])

with open('london_infrastructure/crossing.json', 'r') as f:
    crossing_json = json.loads(f.read())
crossing = pd.json_normalize(crossing_json, record_path = ['features'])

with open('london_infrastructure/cycle_lane_track.json', 'r') as f:
    cycle_lane_track_json = json.loads(f.read())
cycle_lane_track = pd.json_normalize(cycle_lane_track_json, record_path = ['features'])

with open('london_infrastructure/restricted_route.json', 'r') as f:
    restricted_route_json = json.loads(f.read())
restricted_route = pd.json_normalize(restricted_route_json, record_path = ['features'])

In [10]:
# Save as csv files 
advanced_stop_line.to_csv('london_infrastructure/advanced_stop_line.csv', index=False)
crossing.to_csv('london_infrastructure/crossing.csv', index=False)
cycle_lane_track.to_csv('london_infrastructure/cycle_lane_track.csv', index=False)
restricted_route.to_csv('london_infrastructure/restricted_route.csv', index=False)

In [11]:
# Load new csv files
advanced_stop_line = pd.read_csv('london_infrastructure/advanced_stop_line.csv')
crossing = pd.read_csv('london_infrastructure/crossing.csv')
cycle_lane_track = pd.read_csv('london_infrastructure/cycle_lane_track.csv')
restricted_route = pd.read_csv('london_infrastructure/restricted_route.csv')

In [12]:
# Separate list of coordinates into separate rows 
def separate_coordinates(df):
    id_list = []
    coord_list = []
    for i in range(df.shape[0]):
        split_coords = df['geometry.coordinates'][i].split('],')
        for j in range(len(split_coords)):
            id_list.append(df['properties.FEATURE_ID'][i])
            coord_list.append(split_coords[j])
    df_split = pd.DataFrame(zip(id_list, coord_list)).rename(columns={0:'properties.FEATURE_ID', 1:'coordinates'})
    df = pd.merge(df_split, df, on='properties.FEATURE_ID', how='left')
    return df

In [13]:
# Separate coordinates 
crossing = separate_coordinates(crossing)
advanced_stop_line = separate_coordinates(advanced_stop_line)
restricted_route = separate_coordinates(restricted_route)
cycle_lane_track = separate_coordinates(cycle_lane_track)

In [14]:
# Function to process dataframes 

def process_file(file):
    
    # Drop unnecessary columns, rename remaining columns
    file.drop(['type', 'geometry.type', 'geometry.coordinates', 'properties.FEATURE_ID', 
               'properties.PHOTO1_URL', 'properties.PHOTO2_URL'], axis=1, inplace=True)
    file.rename(columns={'properties.SVDATE':'date_of_survey', 'properties.BOROUGH':'borough'}, inplace=True)
    
    # Clean coordinate data
    file['coordinates'] = file['coordinates'].str.replace('[\[\]]', '', regex=True)
    file['latitude'] = [file['coordinates'][i].split(', ')[1] for i in range(file.shape[0])]
    file['longitude'] = [file['coordinates'][i].split(', ')[0] for i in range(file.shape[0])]
    
    # Remove duplicate points (keep most recent)
    file.sort_values(by='date_of_survey', ascending=True, inplace=True)
    file.drop_duplicates(subset=['latitude', 'longitude'], inplace=True, keep='last')
    
    # Return clean file  
    file.drop(['coordinates', 'date_of_survey'], axis=1, inplace=True)
    file.reset_index(drop=True, inplace=True)
    return file

In [15]:
# Process dataframes 
crossing = process_file(crossing)
advanced_stop_line = process_file(advanced_stop_line)
restricted_route = process_file(restricted_route)
cycle_lane_track = process_file(cycle_lane_track)

In [16]:
# Process data further 

# Drop columns with majority null values 
cycle_lane_track.drop(['properties.CLT_ACCESS'], axis=1, inplace=True)

# Set unknown or unexpected values to null  
cycle_lane_track['properties.CLT_SHARED'] = cycle_lane_track['properties.CLT_SHARED'].replace('TCB', np.nan)
cycle_lane_track['properties.CLT_MANDAT'] = cycle_lane_track['properties.CLT_MANDAT'].replace('TCB', np.nan)
cycle_lane_track['properties.CLT_PRIORI'] = cycle_lane_track['properties.CLT_PRIORI'].replace('TRE', np.nan)

In [17]:
# Save final versions 
crossing.to_csv('london_infrastructure/crossing.csv', index=False)
advanced_stop_line.to_csv('london_infrastructure/advanced_stop_line.csv', index=False)
restricted_route.to_csv('london_infrastructure/restricted_route.csv', index=False)
cycle_lane_track.to_csv('london_infrastructure/cycle_lane_track.csv', index=False)

# Create counts by borough

In [18]:
# Assign names to infrastructure dataframes 
restricted_point.attrs['name'] = 'restricted_point'
signal.attrs['name'] = 'signal'
cycle_parking.attrs['name'] = 'cycle_parking'
traffic_calming.attrs['name'] = 'traffic_calming'
signage.attrs['name'] = 'signage'
crossing.attrs['name'] = 'crossing'
advanced_stop_line.attrs['name'] = 'advanced_stop_line'
restricted_route.attrs['name'] = 'restricted_route'
cycle_lane_track.attrs['name'] = 'cycle_lane_track'

In [19]:
# Count the number of each infrastructure surveyed in each borough

borough_df = pd.DataFrame()

for df in [cycle_parking, restricted_point, signage, signal, traffic_calming, crossing, 
           advanced_stop_line, restricted_route, cycle_lane_track]:
    df_count = df.groupby(['borough']).count()[['latitude']].rename(
        columns={'latitude': f"{df.attrs['name']}_count"})
    borough_df = pd.concat([borough_df, df_count], axis=1)

borough_df.fillna(0, inplace=True)
borough_df.reset_index(inplace=True)

In [20]:
# Save to csv
borough_df.to_csv('london_infrastructure/borough_infrastructure.csv', index=False)

# Bike sites data

In [21]:
# Load site data
sites = pd.read_excel('London/Biking sites.xlsx')

In [22]:
# Set up Easting/Northing to Latitude/Longitude converter
transformer = Transformer.from_crs('epsg:27700', 'epsg:4326')

# Get coordinates of sites 
sites['coordinates'] = ''
for i in range(sites.shape[0]):
    sites['coordinates'][i] = transformer.transform(sites.Easting[i], sites.Northing[i])

In [23]:
# Get distances to infrastructure features  

def distance_features(df):

    closest = []
    count_100m = []
    count_1000m = []

    for i in tqdm(range(sites.shape[0])):
        distance = []
        for j in range(df.shape[0]):    
            distance.append(Geod(ellps='WGS84').inv(sites.coordinates[i][1], sites.coordinates[i][0], 
                                             df.longitude[j], df.latitude[j])[2]) 
        closest.append(min(distance))
        count_100m.append(len([m for m in distance if m<=100]))
        count_1000m.append(len([m for m in distance if m<=1000]))

    sites[f"closest_{df.attrs['name']}"] = closest
    sites[f"{df.attrs['name']}_count_100m"] = count_100m
    sites[f"{df.attrs['name']}_count_1000m"] = count_1000m

In [24]:
distance_features(restricted_point)

100%|██████████| 2023/2023 [00:22<00:00, 90.39it/s] 


In [25]:
distance_features(signal)

100%|██████████| 2023/2023 [00:48<00:00, 41.34it/s]


In [26]:
distance_features(crossing)

100%|██████████| 2023/2023 [07:47<00:00,  4.33it/s]


In [27]:
distance_features(advanced_stop_line)

100%|██████████| 2023/2023 [14:09<00:00,  2.38it/s]


In [28]:
distance_features(restricted_route)

100%|██████████| 2023/2023 [1:23:52<00:00,  2.49s/it]   


In [29]:
distance_features(cycle_parking)

100%|██████████| 2023/2023 [5:47:43<00:00, 10.31s/it]     


In [30]:
# Saving this updated version 
sites.to_csv('London/updated_sites.csv', index=False)

# For the future 
- Include traffic_calming, signage and cycle_lane_track 
- Include additional features from each dataframe 